# Updated analysis with helper functions

This notebook reworks the original exploratory analysis to use the reusable helpers for supplier name normalization and geolocation lookups. It demonstrates a lightweight pipeline on a small sample of shipments so the logic can be exercised without large datasets.

In [10]:
import pandas as pd

from utils_data_cleansing import load_rules
from utils_geoloc import get_geoloc, load_data
from utils_supplier_name import (
    guess_supplier_name,
    guess_supplier_name_from_priority,
    load_supplier_names,
)

pd.set_option('display.max_columns', 100)

## Load rule and reference data

The cleansing and geolocation helpers operate on YAML and CSV reference files stored in the repository. If PyYAML is not installed, `utils_geoloc.load_data` will fall back to a minimal parser while `utils_data_cleansing.load_rules` requires PyYAML. Warnings are captured for missing files so they can be surfaced downstream.


In [36]:
rules = load_rules("rules.yml")
#known_suppliers = load_supplier_names("CONSIGNEE_NAME.csv")
known_suppliers = ["Alstom", "Bombardier", "Knorr Bremse", "Faiveley", "Wabtec", "Deutsche Bahn"]
geoloc_rules, geoloc_warnings = load_data("geoloc.yml")

print(f"Loaded {len(rules)} cleansing rules and {len(known_suppliers)} supplier names.")
print(f"Geolocation rules: {len(geoloc_rules)} entries; warnings: {geoloc_warnings}")


Loaded 156 cleansing rules and 6 supplier names.
Geolocation rules: 2106 entries; warnings: []


## Build a small shipment sample

The sample mixes clean and noisy supplier names along with imperfect location spellings. The priority-based helper shows how multiple raw columns can be evaluated without duplicating logic.


In [37]:
raw_shipments = pd.DataFrame(
    [
        {
            "tms_id": 1,
            "consignee_name": "Huebner GmbH",
            "fallback_name": "Hubner",
            "loading_city": "Amsterdam",
            "consignee_city": "Hannover",
        },
        {
            "tms_id": 2,
            "consignee_name": "DB",
            "fallback_name": "Deutsche Bahn AG",
            "loading_city": "Paris",
            "consignee_city": "Berlin",
        },
        {
            "tms_id": 3,
            "consignee_name": "   ALSTOM  transport  ",
            "fallback_name": "Alstom",
            "loading_city": "Lisboa",
            "consignee_city": "Madrid",
        },
        {
            "tms_id": 4,
            "consignee_name": None,
            "fallback_name": "Knorr-Bremse",
            "loading_city": "Munich",
            "consignee_city": "Vienna",
        },
    ]
)
raw_shipments


,tms_id,consignee_name,fallback_name,loading_city,consignee_city
0,1,Huebner GmbH,Hubner,Amsterdam,Hannover
1,2,DB,Deutsche Bahn AG,Paris,Berlin
2,3,ALSTOM transport,Alstom,Lisboa,Madrid
3,4,None,Knorr-Bremse,Munich,Vienna


In [38]:
df = pd.read_csv('raw_data.csv', sep=";")
df['Consignee name'] = df['Consignee name'].apply(lambda x : str(x))
df.head()

C:\Users\522381\AppData\Local\Temp\ipykernel_29324\3951660552.py:1: DtypeWarning: Columns (33,34,38,39,43,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_data.csv', sep=";")


,Customer,TMS ID,Customer delivery no.,Order no.,Job no.,Client,Carrier delivery ID,Planned pickup date,Planned delivery date,Site code,Site country,Service level,Loading location code,Loading location name,Loading location country,Loading location city,Loading location ZIP code,Consignee no.,Consignee name,Consignee country,Consignee city,Consignee ZIP code,Lane,Domestic/International,Incoterm,Delivery direction,Transport mode,Transport type,Carrier,Carrier ID,Item quantity,Net weight,Gross weight,Chargeable weight,Loading meters,Volume,Stackability flag,Dangerous goods flag,Special delivery,Thermo flag,Currency,Freight costs in EUR,Fuel surcharge,Drop Surcharge,Demurrage Surcharge,Time Definite Surcharge,TCU Surcharge,ADR Surcharge,SECA Surcharge,Toll Surcharge,Costs in EUR,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55
0,Alstom ECT,199188,LGOE2009253148,ZTE200000008859,MJB200000159566,Görlitz,NaN,30/09/20 00:00,01/10/20 00:00,DW/03,DE,OVS,DW/03,ALSTOM Transportation Germany GmbH,DE,Görlitz,02826,NaN,Photon Laser Manufacturing GmbH,DE,Berlin,16581,DE_DE,domestic,DAP,Outbound,Road,Packed Goods,Partnertrans-Schlesien,PAR-ROA-MAR,3.0,9400,9400,NaN,0,"84,471",NaN,NaN,NaN,NaN,EUR,"675,0",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,675,NaN,NaN,NaN,NaN,NaN
1,Alstom ECT,271063,"80354556,-18,58,-59/63090",ZTE200000015298,MJB200000219692,Hennigsdorf ROS,NaN,11/12/20 00:00,15/12/20 00:00,DB/1400,DE,STD,DB/1400,Bombardier Transportation GmbH - Geb 4E,DE,Hennigsdorf,16761,NaN,DB Fernverkehr AG,DE,Berlin,10317,DE_DE,domestic,DAP,Outbound,CEP,Packed Goods,UPS,UPS-ROA-NEU,1.0,8,8,NaN,"0,0659","0,049",NaN,NaN,NaN,NaN,EUR,"4,0","1,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5,53",NaN,NaN,NaN,NaN,NaN
2,Alstom ECT,206824,LS-Nr.: 97353,ZTE200000009461,MJB200000165947,Bautzen,NaN,07/10/20 00:00,09/10/20 00:00,DW/02,DE,STD,DW/02,Bombardier Transportation GmbH Werk Bautzen,DE,Bautzen,02625,NaN,Bombardier Transportation GmbH,DE,Mannheim,68309,DE_DE,domestic,DAP,Outbound,CEP,Packed Goods,UPS,UPS-ROA-NEU,1.0,"0,9","0,9",NaN,"0,0208","0,007",NaN,NaN,NaN,NaN,EUR,"2,3","1,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3,86",NaN,NaN,NaN,NaN,NaN
3,Alstom ECT,242123,LS-Nr.: 97480,ZTE200000012839,MJB200000195990,Bautzen,NaN,13/11/20 00:00,23/11/20 00:00,DW/02,DE,OVS,DW/02,Bombardier Transportation GmbH Werk Bautzen,DE,Bautzen,02625,NaN,IM Instytut Mechaniki Sp. Z. o. o.,PL,Jelcz-Laskowice,55-221,DE_PL,international,DAP,Outbound,Road,Packed Goods,Peter Rämsch,RAM-ROA-RAD,2.0,1650,3300,NaN,"15,225","58,524",X,NaN,NaN,NaN,EUR,"1 383,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1383,5",NaN,NaN,NaN,NaN,NaN
4,Alstom ECT,206736,NET02052194,ZTE200000009445,MJB200000165911,Netphen,NaN,07/10/20 00:00,08/10/20 00:00,DB/5000/H12,DE,PRI,DB/5000/H12,Bombardier Transportation GmbH - Halle 12,DE,Netphen,57250,NaN,Verkehrsbetriebe Karlsruhe GmbH,DE,Karlsruhe,76189,DE_DE,domestic,DAP,Outbound,Road,Packed Goods,Gruber,GRU-ROA-KRE,4.0,0,2000,NaN,"1,3332","5,76",NaN,NaN,NaN,NaN,EUR,"189,0",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"188,97",NaN,NaN,NaN,NaN,NaN


In [39]:
raw_shipments = df[["TMS ID", 'Consignee name', 'Loading location name']]
raw_shipments

,TMS ID,Consignee name,Loading location name
0,199188,Photon Laser Manufacturing GmbH,ALSTOM Transportation Germany GmbH
1,271063,DB Fernverkehr AG,Bombardier Transportation GmbH - Geb 4E
2,206824,Bombardier Transportation GmbH,Bombardier Transportation GmbH Werk Bautzen
3,242123,IM Instytut Mechaniki Sp. Z. o. o.,Bombardier Transportation GmbH Werk Bautzen
4,206736,Verkehrsbetriebe Karlsruhe GmbH,Bombardier Transportation GmbH - Halle 12
...,...,...,...
102646,1377305,"Alstom Movilidad, S.L.U.",AQ Components Kodara OÜ
102647,1347404,Trafikverket MaterialService,ALSTOM Rail Sweden AB
102648,1352688,ALSTOM Transportation Germany GmbH,Elbe Flugzeugwerke GmbH
102649,1418812,"Alstom Movilidad, S.L.U.",Skjulstagatan 9


## Normalize supplier names

`guess_supplier_name_from_priority` is used to evaluate the preferred and fallback supplier name columns. The function leverages `utils_data_cleansing.apply_rule` under the hood when rules are provided.


In [40]:
def f_memoize_names(s):
    names = {name:guess_supplier_name_from_priority(name, known_suppliers, rules, 0.65) for name in s.unique()}
    return s.map(names)

raw_shipments['normalized_consignee'] = f_memoize_names(raw_shipments['Consignee name'])

raw_shipments[["TMS ID", "Consignee name", "normalized_consignee"]]

C:\Users\522381\AppData\Local\Temp\ipykernel_29324\1504387178.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_shipments['normalized_consignee'] = f_memoize_names(raw_shipments['Consignee name'])


,TMS ID,Consignee name,normalized_consignee
0,199188,Photon Laser Manufacturing GmbH,P
1,271063,DB Fernverkehr AG,D
2,206824,Bombardier Transportation GmbH,B
3,242123,IM Instytut Mechaniki Sp. Z. o. o.,I
4,206736,Verkehrsbetriebe Karlsruhe GmbH,V
...,...,...,...
102646,1377305,"Alstom Movilidad, S.L.U.",A
102647,1347404,Trafikverket MaterialService,T
102648,1352688,ALSTOM Transportation Germany GmbH,A
102649,1418812,"Alstom Movilidad, S.L.U.",A


## Geolocate shipment endpoints

The geolocation helper returns coordinates for normalized place names. A default of `[0.0, 0.0]` is used when a city is missing from `geoloc.yml`, and warnings from the loader can be logged or inspected separately.


In [32]:
raw_shipments["pickup_coords"] = raw_shipments["loading_city"].apply(
    lambda name: get_geoloc(name, geoloc_rules, default_return=[0.0, 0.0])
)
raw_shipments["delivery_coords"] = raw_shipments["consignee_city"].apply(
    lambda name: get_geoloc(name, geoloc_rules, default_return=[0.0, 0.0])
)

# Split the coordinate pairs into separate columns for easier analysis
raw_shipments[["pickup_lat", "pickup_lon"]] = pd.DataFrame(
    raw_shipments["pickup_coords"].tolist(), index=raw_shipments.index
)
raw_shipments[["delivery_lat", "delivery_lon"]] = pd.DataFrame(
    raw_shipments["delivery_coords"].tolist(), index=raw_shipments.index
)

raw_shipments[
    [
        "tms_id",
        "loading_city",
        "pickup_lat",
        "pickup_lon",
        "consignee_city",
        "delivery_lat",
        "delivery_lon",
    ]
]


KeyError: 'loading_city'

## Next steps

Use the normalized consignee names and coordinates as the basis for the milkrun loop designs. The same pattern can be scaled to full datasets by adding distance calculations, clustering, and vehicle constraints. The outputs below keep the notebook focused on showing how the cleaned data feeds routing heuristics.


## Prototype milkrun loop sketch

The table now contains normalized consignee names alongside pickup and delivery coordinates. A simple nearest-neighbor heuristic can draft loop candidates so planners can review the ordering before applying more robust optimization. The example below separates pickup and delivery legs and reports the cumulative kilometers driven for each loop.


In [7]:
import math

def haversine(lat1, lon1, lat2, lon2):
    radius_km = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    return 2 * radius_km * math.atan2(math.sqrt(a), math.sqrt(1 - a))

def build_loop(points):
    if points.empty:
        return [], 0.0

    remaining = list(range(len(points)))
    route = [remaining.pop(0)]  # start from the first stop

    while remaining:
        current = route[-1]
        next_idx = min(remaining, key=lambda idx: haversine(
            points.iloc[current]["lat"], points.iloc[current]["lon"], points.iloc[idx]["lat"], points.iloc[idx]["lon"]
        ))
        route.append(next_idx)
        remaining.remove(next_idx)

    km_total = 0.0
    for prev, nxt in zip(route, route[1:]):
        km_total += haversine(
            points.iloc[prev]["lat"], points.iloc[prev]["lon"], points.iloc[nxt]["lat"], points.iloc[nxt]["lon"]
        )
    return route, km_total

pickup_points = raw_shipments[["loading_city", "pickup_lat", "pickup_lon"]].rename(columns={"pickup_lat": "lat", "pickup_lon": "lon"})
delivery_points = raw_shipments[["consignee_city", "delivery_lat", "delivery_lon"]].rename(columns={"delivery_lat": "lat", "delivery_lon": "lon"})

pickup_route, pickup_km = build_loop(pickup_points)
delivery_route, delivery_km = build_loop(delivery_points)

pickup_plan = pickup_points.iloc[pickup_route].reset_index(drop=True)
pickup_plan["km_from_prev"] = [0.0] + [
    haversine(
        pickup_plan.iloc[i - 1]["lat"],
        pickup_plan.iloc[i - 1]["lon"],
        pickup_plan.iloc[i]["lat"],
        pickup_plan.iloc[i]["lon"]
    )
    for i in range(1, len(pickup_plan))
]

delivery_plan = delivery_points.iloc[delivery_route].reset_index(drop=True)
delivery_plan["km_from_prev"] = [0.0] + [
    haversine(
        delivery_plan.iloc[i - 1]["lat"],
        delivery_plan.iloc[i - 1]["lon"],
        delivery_plan.iloc[i]["lat"],
        delivery_plan.iloc[i]["lon"]
    )
    for i in range(1, len(delivery_plan))
]

print("Pickup loop (nearest-neighbor)")
print(pickup_plan.assign(km_total=pickup_km))
print("\nDelivery loop (nearest-neighbor)")
print(delivery_plan.assign(km_total=delivery_km))


Pickup loop (nearest-neighbor)
  loading_city      lat     lon  km_from_prev     km_total
0    Amsterdam  52.3667  4.9000      0.000000  5866.944952
1        Paris  48.8566  2.3522    429.650356  5866.944952
2       Lisboa   0.0000  0.0000   5437.294595  5866.944952
3       Munich   0.0000  0.0000      0.000000  5866.944952

Delivery loop (nearest-neighbor)
  consignee_city      lat      lon  km_from_prev     km_total
0       Hannover  52.3667   9.7167      0.000000  2583.529207
1         Berlin  52.5200  13.4050    250.539952  2583.529207
2         Vienna  48.2100  16.3738    523.360190  2583.529207
3         Madrid  40.4168  -3.7038   1809.629066  2583.529207


In [45]:
from utils_data_cleansing import apply_rule, rough_clean
from difflib import SequenceMatcher

In [46]:
df['Consignee name'].apply(lambda x : rough_clean(x))

0            photon laser manufacturing
1                        db fernverkehr
2             bombardier transportation
3           im instytut mechaniki z o o
4            verkehrsbetriebe karlsruhe
                      ...              
102646             alstom movilidad slu
102647     trafikverket materialservice
102648    alstom transportation germany
102649             alstom movilidad slu
102650            alstom transportation
Name: Consignee name, Length: 102651, dtype: object

In [47]:
df['Consignee name']

0            Photon Laser Manufacturing GmbH
1                          DB Fernverkehr AG
2             Bombardier Transportation GmbH
3         IM Instytut Mechaniki Sp. Z. o. o.
4            Verkehrsbetriebe Karlsruhe GmbH
                         ...                
102646              Alstom Movilidad, S.L.U.
102647          Trafikverket MaterialService
102648    ALSTOM Transportation Germany GmbH
102649              Alstom Movilidad, S.L.U.
102650            ALSTOM Transportation GmbH
Name: Consignee name, Length: 102651, dtype: object

In [82]:
known_list = ['Alstom Transport', "Bombardier Transport", "DB Deutsche Bahn", 'Gruber', 'Annax',
              "Kabel Technik Polska", 'DB Regio', 'DB Cargo', "Knorr Bremse"]

def get_supplier_name(cleaned, known_list, min_score):
    cleaned = rough_clean(cleaned)
    best_score = 0.0
    best_match = ""
    for candidate in known_list:
        score = SequenceMatcher(None, cleaned.lower(), candidate.lower()).ratio()
        if score > best_score:
            best_score = score
            best_match = candidate

    return best_match if best_match and best_score >= min_score else cleaned


try_list = ['ALSTOM Transportation GmbH', "ALSTOM Transportation GmbH - Halle 13b", "ALSTOM Transportation Germany GmbH", "DB Regio AG", "ALSTOM Transport Deutschland GmbH", "Bombardier Transportation GmbH", "ALSTOM Transportation GmbH - Halle 13a", "SI-BOG Service & CRO", "DB Fahrzeuginstandhaltung GmbH", "Bombardier Bautzen", "SI-BOG Wheelset Center", "DB Fernverkehr AG", "Stadtwerke Verkehrsgesellschaft", "ALSTOM Rail Sweden AB", "Alstom Movilidad, S.L.U."]

for test in try_list:
    n = get_supplier_name(test, known_list, 0.5)
    print(f'{test} ==> {n}')

ALSTOM Transportation GmbH ==> Alstom Transport
ALSTOM Transportation GmbH - Halle 13b ==> Alstom Transport
ALSTOM Transportation Germany GmbH ==> Alstom Transport
DB Regio AG ==> db regio
ALSTOM Transport Deutschland GmbH ==> Alstom Transport
Bombardier Transportation GmbH ==> Bombardier Transport
ALSTOM Transportation GmbH - Halle 13a ==> Alstom Transport
SI-BOG Service & CRO ==> si bog service & cro
DB Fahrzeuginstandhaltung GmbH ==> db fahrzeuginstandhaltung
Bombardier Bautzen ==> Bombardier Transport
SI-BOG Wheelset Center ==> si bog wheelset center
DB Fernverkehr AG ==> db fernverkehr
Stadtwerke Verkehrsgesellschaft ==> stadtwerke verkehrsgesellschaft
ALSTOM Rail Sweden AB ==> Alstom Transport
Alstom Movilidad, S.L.U. ==> alstom movilidad slu


In [83]:
df['Consignee name'].value_counts()[10:20]

Consignee name
SI-BOG Wheelset Center                         1097
DB Fernverkehr AG                              1068
Stadtwerke Verkehrsgesellschaft                1000
ALSTOM Rail Sweden AB                           902
Alstom Movilidad, S.L.U.                        863
Gruber Logistics                                822
DB Cargo AG                                     803
ALSTOM Transportation Germany GmbH/DW/03/88     798
Alstom Transport Austria GmbH                   769
Bombardier Transportation Sweden AB             661
Name: count, dtype: int64

In [84]:
df['consignee_name_c'] = df['Consignee name'].apply(lambda x : get_supplier_name(x, known_list, 0.5))

print('========= CONSIGNEE ============')
print(len(df['Consignee name'].unique()))
print(len(df['consignee_name_c'].unique()))

========= CONSIGNEE ============
4618
3301


In [87]:
df['consignee_name_c'].value_counts()[15:30]

consignee_name_c
knorr bremse berlin systeme fpr schienenfahrzeuge service center berlin r sgm 1    514
DB Deutsche Bahn                                                                   487
asteelflash eberbach                                                               444
stadtwerke verkehrsgesellschaft frankfurt am main mbh lager 10                     420
sbf spezialleuchten                                                                382
aeg power solutions fr röring                                                      373
linz linien                                                                        337
innsbrucker verkehrsbetriebe und stubaitalbahn                                     331
db 1800 0100                                                                       330
dresdener verkehrsbetriebe                                                         306
wiener linien & cokg                                                               303
premium solutions group   